In [11]:
import os
import cv2
import logging

# Set up basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def calculate_brightness(image):
    """Calculate the average brightness of a grayscale version of the image."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return np.mean(gray)

def classify_image_by_brightness(brightness, thresholds):
    """Classify an image based on its brightness using predefined thresholds."""
    if brightness >= thresholds['well_lit']:
        return 'well_lit'
    elif brightness >= thresholds['normal']:
        return 'normal'
    else:
        return 'no_light'

def classify_and_store_images(source_directory, target_directory, thresholds, img_dim=(256, 256)):
    """Process images from the source directory, classify them by brightness, and store them in the target directory."""
    if not os.path.exists(target_directory):
        os.makedirs(target_directory)
    
    for root, _, files in os.walk(source_directory):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                file_path = os.path.join(root, file)
                img = cv2.imread(file_path)
                if img is not None:
                    img_resized = cv2.resize(img, img_dim)
                    brightness = calculate_brightness(img_resized)
                    category = classify_image_by_brightness(brightness, thresholds)
                    category_path = os.path.join(target_directory, category)
                    os.makedirs(category_path, exist_ok=True)
                    cv2.imwrite(os.path.join(category_path, file), img_resized)
                    logging.info(f"Processed, classified as {category}, and saved: {os.path.join(category_path, file)}")
                else:
                    logging.error(f"Failed to read image: {file_path}")


In [15]:
# Set brightness thresholds for each category
brightness_thresholds = {
    'well_lit': 150,  # Brightness level above which images are considered well-lit
    'normal': 100,    # Brightness level above which images are considered normal
    'no_light': 50    # Brightness level below which images are considered to have no light
}

# Directories
source_directory = '/Users/hima/Desktop/twinverse/lol'
target_directory = '/Users/hima/Desktop/twinverse/classify'

# Process and classify images based on the defined thresholds
classify_and_store_images(source_directory, target_directory, brightness_thresholds)


2025-03-15 15:12:03,253 - INFO - Processed, classified as normal, and saved: /Users/hima/Desktop/twinverse/classify/normal/638.png
2025-03-15 15:12:03,269 - INFO - Processed, classified as no_light, and saved: /Users/hima/Desktop/twinverse/classify/no_light/176.png
2025-03-15 15:12:03,296 - INFO - Processed, classified as normal, and saved: /Users/hima/Desktop/twinverse/classify/normal/88.png
2025-03-15 15:12:03,302 - INFO - Processed, classified as normal, and saved: /Users/hima/Desktop/twinverse/classify/normal/610.png
2025-03-15 15:12:03,308 - INFO - Processed, classified as normal, and saved: /Users/hima/Desktop/twinverse/classify/normal/604.png
2025-03-15 15:12:03,314 - INFO - Processed, classified as normal, and saved: /Users/hima/Desktop/twinverse/classify/normal/162.png
2025-03-15 15:12:03,319 - INFO - Processed, classified as normal, and saved: /Users/hima/Desktop/twinverse/classify/normal/189.png
2025-03-15 15:12:03,325 - INFO - Processed, classified as normal, and saved: /Us

In [26]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_and_train_model(data_dir):
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    train_gen = datagen.flow_from_directory(
        data_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical',
        subset='training'
    )
    val_gen = datagen.flow_from_directory(
        data_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical',
        subset='validation'
    )

    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_gen, validation_data=val_gen, epochs=10)
    model.save('model_path/light_classification_model.h5')
    return model

# Path to the directory where images are stored
data_directory = '/Users/hima/Desktop/twinverse/classify'
create_and_train_model(data_directory)


Found 440 images belonging to 3 classes.
Found 110 images belonging to 3 classes.
Epoch 1/10


/opt/anaconda3/envs/twinverse/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/anaconda3/envs/twinverse/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 156ms/step - accuracy: 0.4944 - loss: 1.6864 - val_accuracy: 0.6818 - val_loss: 0.8105
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 139ms/step - accuracy: 0.7484 - loss: 0.6428 - val_accuracy: 0.7091 - val_loss: 0.7119
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 139ms/step - accuracy: 0.8206 - loss: 0.4454 - val_accuracy: 0.7091 - val_loss: 0.7220
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 139ms/step - accuracy: 0.8571 - loss: 0.3414 - val_accuracy: 0.7091 - val_loss: 0.6540
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 144ms/step - accuracy: 0.8687 - loss: 0.2769 - val_accuracy: 0.7818 - val_loss: 0.4455
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 153ms/step - accuracy: 0.8987 - loss: 0.2457 - val_accuracy: 0.6545 - val_loss: 0.7134
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 161ms/step - accuracy: 0.9180 - loss: 0.1795 - val_accuracy: 0.7636 - val_loss: 0.5028
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 169ms/step - accuracy: 0.9122 - loss: 0.1759 - val_accuracy: 0.7091 - val_

2025-03-15 16:06:58,547 - WARNING - You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 


<Sequential name=sequential_5, built=True>